In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow

     |████████████████████████████████| 332.1MB 66kB/s 
     |████████████████████████████████| 3.0MB 25.1MB/s 
     |████████████████████████████████| 419kB 43.9MB/s 


In [0]:
import os
os.chdir(r"/content/gdrive/My Drive/Colab Notebooks/GAN")

In [0]:
#Imorting the libraries
from __future__ import print_function, division

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD

import matplotlib.pyplot as plt

import numpy as np

#Creating a class.
class CGAN():
  #Lets Initilized all necessary variables
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = SGD(lr=0.1, momentum=0.5, decay=1.00004)

        # Build the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        self.generator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

       
        noise = Input(shape=(100,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Pass generated image as input and determines validity to the descriminitor
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

    def build_generator(self):
        #Take the noise and label (0.9) 
        
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))
        model_input = multiply([noise, label_embedding])
        
        print(noise.shape)
        print(label_embedding.shape)
        model = Sequential()

        model.add(Dense(200, input_dim=self.latent_dim))
        model.add(Activation('relu'))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(1000))
        model.add(Activation('relu'))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(1200, input_dim=self.latent_dim))
        model.add(Activation('relu'))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dropout(0.5))

        model.add(Dense(np.prod(self.img_shape), activation='sigmoid'))
        model.add(Reshape(self.img_shape))

        model.summary()

        img = model(model_input)
        
        return Model([noise, label], img)

    def build_discriminator(self):
        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)
        model_input = multiply([flat_img, label_embedding])

        model = Sequential()

        model.add(Dense(240,  input_dim=np.prod(self.img_shape)))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))

        model.add(Dense(50))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))

        model.add(Dense(240))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))

        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        validity = model(model_input)

        return Model([img, label], validity)

    def train(self, epochs, batch_size=100, save_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)


        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs, labels = X_train[idx], y_train[idx]

            noise = np.random.normal(0, 1, (batch_size, 100))

            gen_imgs = self.generator.predict([noise, labels])

            valid = np.ones((batch_size, 1))
            fake = np.zeros((batch_size, 1))

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
            
            d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
            
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            valid = np.ones((batch_size, 1))
            # Generator wants discriminator to label the generated images as the intended
            # digits
            sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)
            # print("g loss")
            # print(g_loss)
            # print("")

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 2, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.arange(0, 10).reshape(-1, 1)

        gen_imgs = self.generator.predict([noise, sampled_labels])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        fig.suptitle("CGAN: Generated digits", fontsize=12)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].set_title("Digit: %d" % sampled_labels[cnt])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    cgan = CGAN()
    cgan.train(epochs=600, batch_size=32, save_interval=200)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 240)               188400    
_________________________________________________________________
activation (Activation)      (None, 240)               0         
_________________________________________________________________
dropout (Dropout)            (None, 240)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                12050     
_________________________________________________________________
activation_1 (Activation)    (None, 50)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 240)               1

W0621 09:09:45.647927 140440944097152 training.py:2131] Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
W0621 09:09:46.877578 140440944097152 training.py:2131] Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?


0 [D loss: 0.718670, acc.: 7.81%] [G loss: 0.659844]
1 [D loss: 0.698599, acc.: 48.44%] [G loss: 0.655864]
2 [D loss: 0.697209, acc.: 48.44%] [G loss: 0.657297]
3 [D loss: 0.696475, acc.: 50.00%] [G loss: 0.652890]
4 [D loss: 0.696979, acc.: 50.00%] [G loss: 0.657061]
5 [D loss: 0.694303, acc.: 50.00%] [G loss: 0.655903]
6 [D loss: 0.694201, acc.: 50.00%] [G loss: 0.656350]
7 [D loss: 0.692966, acc.: 50.00%] [G loss: 0.659948]
8 [D loss: 0.693289, acc.: 50.00%] [G loss: 0.657782]
9 [D loss: 0.693350, acc.: 48.44%] [G loss: 0.655864]
10 [D loss: 0.693066, acc.: 50.00%] [G loss: 0.655937]
11 [D loss: 0.694203, acc.: 50.00%] [G loss: 0.658318]
12 [D loss: 0.693442, acc.: 50.00%] [G loss: 0.660178]
13 [D loss: 0.693760, acc.: 50.00%] [G loss: 0.656997]
14 [D loss: 0.695801, acc.: 50.00%] [G loss: 0.654951]
15 [D loss: 0.691760, acc.: 50.00%] [G loss: 0.661331]
16 [D loss: 0.694819, acc.: 50.00%] [G loss: 0.654748]
17 [D loss: 0.693929, acc.: 48.44%] [G loss: 0.659709]
18 [D loss: 0.694688,